In [1]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import re
import datetime

In [2]:
#검색어 입력
k='치즈볼'

In [24]:
df = pd.read_csv('./crawling/facebook{}.csv'.format(k), index_col=0)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296 entries, 0 to 295
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    296 non-null    object
 1   title   255 non-null    object
dtypes: object(2)
memory usage: 6.9+ KB
None


In [25]:
df1 = df.dropna(subset=['title'], how='any', axis=0) #title의 결측치 행 삭제
df1.reset_index(inplace=True, drop=True)
print(len(df1))
print(df1.info())

255
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    255 non-null    object
 1   title   255 non-null    object
dtypes: object(2)
memory usage: 4.1+ KB
None


In [26]:
print(df1)

                date                                              title
0              9월 1일  그 안에 무엇을 넣고 돌려도 겉바속촉 명품 요리로 변신시켜주는 주방의 요물 #에어프...
1    10월 11일 오후 6:00  광희가 네고하고 간 바로 그 bbq신메뉴 메이플버터갈릭치킨 와우 와우 와우 ! 달달...
2    10월 12일 오후 1:44                                  치즈볼 맛있게 먹기~~ ㅋㅋ ㆍ
3     10월 9일 오전 1:32                                                  ·
4     9월 24일 오후 6:10  Bhc 사이드메뉴에게 바치는 노래 너와 결혼까지 생각했어~ 같은 집 같은 방에서.....
..               ...                                                ...
250   9월 15일 오후 4:34             순수치킨이 새로 쓰는 #치킨의정석  [가성비+가성비=갓성비=순수치킨]
251      2016년 1월 8일  요즘 핫한 맘스터치 사이드메뉴~ 이 중에서 가장 좋아하는 최애메뉴는? #올해도_다이...
252     2018년 9월 10일  주말이 순삭되었다 우울한 내게 처방전은 #치즈볼 이지 쫀독쫀독하고 바삭한 치즈볼에 ...
253      2018년 8월 2일                치즈 덕후는 꼭 시킨다는 #BHC #치즈볼 0:00 / 0:05
254      2018년 8월 2일                치즈 덕후는 꼭 시킨다는 #BHC #치즈볼 0:02 / 0:05

[255 rows x 2 columns]


In [27]:
#2020년 이전 데이터 none처리, 시간정보만 삭제
for i in range(len(df1['date'])):
    if '년' in df1['date'][i]:
        df1['date'][i] = None
    elif '함께 합니다' in df1['date'][i]:
        df1['date'][i] = None
    elif '오' in df1['date'][i]:
        df1['date'][i] = df1['date'][i][:(df1['date'][i].find('오'))-1]
    print(df1['date'][i])

9월 1일
10월 11일
10월 12일
10월 9일
9월 24일
9월 22일
9월 30일
10월 2일
9월 24일
8월 16일
9월 7일
8월 13일
9월 29일
10월 11일
10월 11일
6월 1일
9월 26일
6시간
10월 3일
9월 24일
9월 17일
10월 5일
9월 15일
9월 17일
9월 28일
9월 26일
9월 17일
5월 19일
8월 2일
1월 31일
None
None
3월 10일
1월 14일
None
None
None
None
None
None
None
10월 3일
9월 21일
None
10월 10일
6월 29일
None
15시간
None
None
5월 31일
None
None
None
10월 10일
9월 29일
10월 7일
9월 17일
10월 5일
10월 9일
10월 9일
9월 30일
10월 7일
None
9월 21일
9월 27일
10월 9일
10월 11일
9월 18일
10월 7일
10월 6일
9월 23일
9월 22일
9월 22일
9월 15일
10월 10일
10월 10일
10월 4일
10월 1일
6월 20일
9월 25일
2월 14일
8월 9일
9월 24일
9월 21일
10월 7일
9월 21일
9월 13일
8월 24일
10월 10일
9월 18일
9월 17일
8월 23일
9월 21일
9월 29일
6월 16일
5월 20일
5월 14일
5월 9일
1월 13일
2월 6일
None
None
None
8월 29일
None
None
None
None
5월 30일
None
None
10월 9일
9월 29일
9월 23일
9월 30일
9월 23일
9월 21일
10월 2일
9월 26일
9월 18일
9월 22일
9월 17일
10월 5일
어제
9월 27일
9월 16일
9월 22일
9월 22일
None
9월 21일
9월 29일
9월 16일
None
7월 19일
None
9월 28일
7월 31일
9월 16일
None
None
3월 17일
2월 19일
9월 18일
9월 22일
None
None
None
9월 17일
9월 21일
None
None
10월 3일
9월 2일
6

C:\ProgramData\Anaconda3\envs\MachineLearning\lib\site-packages\IPython\core\interactiveshell.py:3417: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
df_clean = df1.dropna(subset=['date'], how='any', axis=0) #2020년 이전 행 삭제
df_clean.reset_index(inplace=True, drop=True)
print(len(df_clean))
print(df_clean.info())

174
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    174 non-null    object
 1   title   174 non-null    object
dtypes: object(2)
memory usage: 2.8+ KB
None


In [29]:
#2020년 추가
for i in range(len(df_clean['date'])):
    df_clean['date'][i]='2020년 '+df_clean['date'][i]
print(df_clean['date'])

0        2020년 9월 1일
1      2020년 10월 11일
2      2020년 10월 12일
3       2020년 10월 9일
4       2020년 9월 24일
           ...      
169     2020년 1월 28일
170     2020년 9월 18일
171     2020년 9월 25일
172      2020년 6월 5일
173     2020년 9월 15일
Name: date, Length: 174, dtype: object


C:\ProgramData\Anaconda3\envs\MachineLearning\lib\site-packages\IPython\core\interactiveshell.py:3417: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
#년월일을 - 로 대체
for i in range(len(df_clean)):
    df_clean['date'][i] = df_clean['date'][i].replace('년 ', '-').replace('월 ', '-').replace('일', '')
print(df_clean)    

           date                                              title
0      2020-9-1  그 안에 무엇을 넣고 돌려도 겉바속촉 명품 요리로 변신시켜주는 주방의 요물 #에어프...
1    2020-10-11  광희가 네고하고 간 바로 그 bbq신메뉴 메이플버터갈릭치킨 와우 와우 와우 ! 달달...
2    2020-10-12                                  치즈볼 맛있게 먹기~~ ㅋㅋ ㆍ
3     2020-10-9                                                  ·
4     2020-9-24  Bhc 사이드메뉴에게 바치는 노래 너와 결혼까지 생각했어~ 같은 집 같은 방에서.....
..          ...                                                ...
169   2020-1-28  연휴가 끝나서 힘들고 지칠 때 #멕시카나치킨 을 찾아주세요~! 매콤단짠  #뿌리고치...
170   2020-9-18                                배달의민족 순수치킨 3000원 할인
171   2020-9-25                          금요일은 배달의민족 순수치킨  3000원 할인
172    2020-6-5  #또래오래 #목우촌 #배달의민족 #쿠폰 #프로모션 #치킨 이번 주 금토일 치킨파티 ...
173   2020-9-15             순수치킨이 새로 쓰는 #치킨의정석  [가성비+가성비=갓성비=순수치킨]

[174 rows x 2 columns]


In [31]:
#시간, 어제 - 날짜형식으로 입력
for z in range(len(df_clean['date'])):
    if '시간' in df_clean['date'][z]:
        new_date = datetime.datetime.now() - datetime.timedelta(days = 1)            
        df_clean['date'][z] = new_date
        
    elif '어제' in df_clean['date'][z]:
        new_date2 = datetime.datetime.now() - datetime.timedelta(days = 1)
        df_clean['date'][z] = new_date2
    
    else:
        pass
print(df_clean['date'])

0        2020-9-1
1      2020-10-11
2      2020-10-12
3       2020-10-9
4       2020-9-24
          ...    
169     2020-1-28
170     2020-9-18
171     2020-9-25
172      2020-6-5
173     2020-9-15
Name: date, Length: 174, dtype: object


C:\ProgramData\Anaconda3\envs\MachineLearning\lib\site-packages\IPython\core\interactiveshell.py:3417: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [32]:
#전체 데이터 날짜 형식으로 변환
df_clean['date'] = pd.to_datetime(df_clean['date'])

C:\ProgramData\Anaconda3\envs\MachineLearning\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
#날짜 데이터로 변환
for i in range(len(df_clean)):
    df_clean['date'][i] = df_clean['date'][i].strftime('%Y-%m-%d')
df_clean['date']

C:\ProgramData\Anaconda3\envs\MachineLearning\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\MachineLearning\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\envs\MachineLearning\lib\site-packages\pandas\core\series.py:1061: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

0      2020-09-01
1      2020-10-11
2      2020-10-12
3      2020-10-09
4      2020-09-24
          ...    
169    2020-01-28
170    2020-09-18
171    2020-09-25
172    2020-06-05
173    2020-09-15
Name: date, Length: 174, dtype: object

In [34]:
df_clean                                    

,date,title
0,2020-09-01,그 안에 무엇을 넣고 돌려도 겉바속촉 명품 요리로 변신시켜주는 주방의 요물 #에어프...
1,2020-10-11,광희가 네고하고 간 바로 그 bbq신메뉴 메이플버터갈릭치킨 와우 와우 와우 ! 달달...
2,2020-10-12,치즈볼 맛있게 먹기~~ ㅋㅋ ㆍ
3,2020-10-09,·
4,2020-09-24,Bhc 사이드메뉴에게 바치는 노래 너와 결혼까지 생각했어~ 같은 집 같은 방에서.....
...,...,...
169,2020-01-28,연휴가 끝나서 힘들고 지칠 때 #멕시카나치킨 을 찾아주세요~! 매콤단짠 #뿌리고치...
170,2020-09-18,배달의민족 순수치킨 3000원 할인
171,2020-09-25,금요일은 배달의민족 순수치킨 3000원 할인
172,2020-06-05,#또래오래 #목우촌 #배달의민족 #쿠폰 #프로모션 #치킨 이번 주 금토일 치킨파티 ...


In [35]:
df_clean.to_csv('./crawling/facebook{}_clean.csv'.format(k))